In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import urllib.request, urllib.parse, urllib.error


#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}


s = requests.Session()
s.headers['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36'
r = s.get("https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc")

#getting all the hyperlinks and string as text. Sll yhr 500 links from the fisrt webpage
all_links = []
for i in range(5):
    req = s.get('https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(i))
    c=req.content
    #fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(c, 'html.parser') 
    hyperlinks = soup.find_all('a', attrs = {'class':'title'})
    all_links.extend(hyperlinks)

#the list contains all 500 movies pulling out the links from the second webpage(containing director and genre of each movie)
links = []
for i in all_links:
    links.append('https://www.metacritic.com' + i.get('href'))
links[:5]

#pulling movie names from the links and putting it into a list
movies=[]
for i in range(5):
    req = s.get('https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page='+str(i))
    c = req.content
    soup = BeautifulSoup(c, 'html.parser') 
    z = soup.find_all("td",{"class":"clamp-summary-wrap"})
    for j in range(len(z)):
        movies.append(z[j].find("h3").text)
        
    
directors=[]
genres=[]
for j in links:  
    #print(j)
    r2 = s.get(j)
    c2=r2.content
    soup2 = BeautifulSoup(c2,"html.parser")
    
    director=soup2.find("div",{"class":"director"})
    director_a = director.find_all("a")
    dirt = []
    for y in director_a:
        dirt.append(y.get_text())    
    dirt = tuple(dirt)

   
    genre=soup2.find("div",{"class":"genres"}).find_all("span")[2:]
    
    for i in range(len(genre)):
        genre[i] = genre[i].text    
    genres.append(genre)
    directors.append(dirt)



In [2]:
sub_dict={}
for i in range(len(directors)):
    sub_dict[directors[i]]=genres[i]

In [3]:
sub_dict

{('Francis Ford Coppola',): ['Drama', 'Mystery', 'Thriller'],
 ('Orson Welles',): ['Drama', 'History', 'Comedy', 'War'],
 ('Alfred Hitchcock',): ['Thriller', 'Crime', 'Film-Noir'],
 ('Michael Curtiz',): ['Drama', 'Mystery', 'Romance', 'Crime', 'Film-Noir'],
 ('Richard Linklater',): ['Drama', 'Romance'],
 ('Krzysztof Kieslowski',): ['Drama', 'Comedy', 'Romance'],
 ('Gene Kelly', 'Stanley Donen'): ['Comedy', 'Romance', 'Musical'],
 ('Charles Chaplin',): ['Comedy', 'Romance'],
 ('Barry Jenkins',): ['Drama', 'Romance', 'Crime'],
 ('D.W. Griffith',): ['Drama', 'History'],
 ('Ben Sharpsteen',
  'Bill Roberts',
  'Hamilton Luske',
  'Jack Kinney',
  'Norman Ferguson',
  'T. Hee',
  'Wilfred Jackson'): ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical'],
 ('Akira Kurosawa',): ['Adventure', 'Drama'],
 ('Robert Bresson',): ['Drama', 'Crime'],
 ('John Huston',): ['Drama', 'Sport'],
 ('Guillermo del Toro',): ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War'],
 ('Billy Wilder',): ['Drama', 'Com


director = []  
genres = []
titles = []
for i in all_urls:
    req = urllib.request.Request(i, headers = headers )
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser')
    
    titles = []
for i in all_urls:
    #titles.append(i.get_text()) 
    req = urllib.request.Request('https://www.metacritic.com/movie/', headers = headers)
    fhand = urllib.request.urlopen(req)
    soup = BeautifulSoup(fhand, 'html.parser') 
    title = soup.find_all('div', attrs = {'class':'product_page_title oswald'})
    #titles.append(i.text)
    
    #Director 
    director_div = soup.find('div',attrs ={'class':'director'} )
    director_link = director_div.find_all('a')
    for y in director_link:
        director.append(y.text)
        for y in director:
            director_tupple = tuple(director)
    
    # Genre 
    genre_div = soup.find('div' , attrs = {'class':'genres'})
    genre_list = genre_div.find_all('span')
    gen_list = []
    for j in genre_list:
        gen_list.append(j.get_text())
    genres.append(gen_list[2:])
    



In [4]:
df = pd.DataFrame(list(zip(movies, directors, genres)),
               columns =['Movie', 'Director', 'Genre'])
df

,Movie,Director,Genre
0,The Godfather,"(Francis Ford Coppola,)","[Drama, Thriller, Crime]"
1,Citizen Kane,"(Orson Welles,)","[Drama, Mystery]"
2,Rear Window,"(Alfred Hitchcock,)","[Mystery, Thriller]"
3,Casablanca,"(Michael Curtiz,)","[Drama, Romance, War]"
4,Boyhood,"(Richard Linklater,)",[Drama]
5,Three Colors: Red,"(Krzysztof Kieslowski,)","[Drama, Mystery, Romance]"
6,Vertigo,"(Alfred Hitchcock,)","[Mystery, Thriller, Romance]"
7,Notorious,"(Alfred Hitchcock,)","[Drama, Thriller, Romance, Film-Noir]"
8,Singin' in the Rain,"(Gene Kelly, Stanley Donen)","[Comedy, Romance, Musical]"
9,City Lights,"(Charles Chaplin,)","[Drama, Comedy, Romance]"


In [97]:
dict500 = {}
for i in range(len(movies)):
    #movie_name = i.text
    #get_directors
    #get genres
    print(genres[2:])
    sub_dict = {}
    sub_dict[directors[i]] = genres[i]
    dict500[movies[i]] = sub_dict

[['Mystery', 'Thriller'], ['Drama', 'Romance', 'War'], ['Drama'], ['Drama', 'Mystery', 'Romance'], ['Mystery', 'Thriller', 'Romance'], ['Drama', 'Thriller', 'Romance', 'Film-Noir'], ['Comedy', 'Romance', 'Musical'], ['Drama', 'Comedy', 'Romance'], ['Drama'], ['Drama', 'History'], ['Fantasy', 'Comedy', 'Animation', 'Family', 'Musical'], ['Drama', 'Thriller', 'Crime', 'Film-Noir'], ['Action', 'Adventure', 'Drama'], ['Drama', 'Crime'], ['Mystery', 'Thriller'], ['Adventure', 'Drama', 'Western'], ['Drama', 'Mystery', 'Thriller', 'Fantasy', 'War'], ['Comedy', 'Romance'], ['Action', 'Adventure', 'Mystery', 'Thriller'], ['Drama', 'Sport', 'Documentary'], ['Drama', 'Mystery', 'Crime'], ['Drama'], ['Drama', 'Romance'], ['Action', 'Adventure', 'Western'], ['Biography', 'Drama'], ['Mystery', 'Thriller', 'Film-Noir'], ['Thriller', 'Comedy'], ['Drama', 'History', 'War'], ['Mystery', 'Thriller', 'Horror'], ['Drama', 'History', 'Romance', 'War'], ['Drama'], ['Drama', 'History', 'Thriller', 'War'], ['D

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [66]:
df = {'Movie': movies, 'Director': directors, 'Genre':genres}
data = pd.DataFrame(df)
data

,Movie,Director,Genre
0,The Godfather,"(Francis Ford Coppola,)","[Drama, Thriller, Crime]"
1,Citizen Kane,"(Orson Welles,)","[Drama, Mystery]"
2,Rear Window,"(Alfred Hitchcock,)","[Mystery, Thriller]"
3,Casablanca,"(Michael Curtiz,)","[Drama, Romance, War]"
4,Boyhood,"(Richard Linklater,)",[Drama]
5,Three Colors: Red,"(Krzysztof Kieslowski,)","[Drama, Mystery, Romance]"
6,Vertigo,"(Alfred Hitchcock,)","[Mystery, Thriller, Romance]"
7,Notorious,"(Alfred Hitchcock,)","[Drama, Thriller, Romance, Film-Noir]"
8,Singin' in the Rain,"(Gene Kelly, Stanley Donen)","[Comedy, Romance, Musical]"
9,City Lights,"(Charles Chaplin,)","[Drama, Comedy, Romance]"


In [67]:
len(dict500)

497

In [99]:
# saving the dataframe
df.to_csv('Metacritics.csv')


## Taking Movie name as input

In [69]:
movie_name=input("Enter the movie_name:")
print(dict500[movie_name])

Enter the movie_nameSnow White and the Seven Dwarfs
{('Ben Sharpsteen', 'David Hand', 'Larry Morey', 'Perce Pearce', 'Wilfred Jackson', 'William Cottrell'): ['Fantasy', 'Romance', 'Animation', 'Family', 'Musical']}


In [70]:
## need to look into formatting. All genres must appear in one line.
for key,value in dict500[movie_name].items():
    for i in key:
        print(f'The diretor of the movie is {i}')
    for j in value:
        print(f'The genre of the movie is {j}')

The diretor of the movie is Ben Sharpsteen
The diretor of the movie is David Hand
The diretor of the movie is Larry Morey
The diretor of the movie is Perce Pearce
The diretor of the movie is Wilfred Jackson
The diretor of the movie is William Cottrell
The genre of the movie is Fantasy
The genre of the movie is Romance
The genre of the movie is Animation
The genre of the movie is Family
The genre of the movie is Musical


##  Operation based on user input 

In [102]:
inp1 = input("What do you want to check?")
if inp1 == 'director':
    inp = input("Enter the name of the director")

What do you want to check?director
Enter the name of the directorAnurag Kashyap


In [103]:
movies_directed = []
genres_directed = []
for key1, value1 in dict500.items():
    for key2, value2 in value1.items():
        if inp in key2:
            movies_directed.append(key1)
            genres_directed.extend(value2)

In [104]:
#eliminate every last element in the string and replace with '.'
# do it outside the for loop.
s=''
for i in movies_directed:
    s = s + i +', '
print(inp,'has directed',s)

Anurag Kashyap has directed Gangs of Wasseypur, 


In [105]:
genres_freq={}
for i in genres_directed:
    if i in genres_freq:
        genres_freq[i]+=1
    else:
        genres_freq[i]=1

In [106]:
genres_freq

{'Action': 1, 'Drama': 1, 'Thriller': 1, 'Crime': 1}

In [107]:
s=''
for key,value in genres_freq.items():
    s = s + " " + key + ":" + " " + str(value)
    
print(f'His most directed Genres are {s}')

His most directed Genres are  Action: 1 Drama: 1 Thriller: 1 Crime: 1


In [109]:
director1 = input("Enter name of the first director")
director2 = input("Enter name of the second director")
genres_1 = []

for key1, value1 in dict500.items():
    for key2, value2 in value1.items():
        if director1 in key2:
            #movies_directed.append(key1)
            genres_1.extend(value2)
genres_2 = []

for key1, value1 in dict500.items():
    for key2, value2 in value1.items():
        if director2 in key2:
            #movies_directed.append(key1)
            genres_2.extend(value2)
        

Enter name of the first directorChristopher Nolan
Enter name of the second directorAnurag Kashyap


In [110]:
genres_1

['Action', 'Drama', 'History', 'Thriller', 'War']

In [111]:
genres_2

['Action', 'Drama', 'Thriller', 'Crime']

In [45]:
common = set(genres_1).union(genres_2)
common

{'Action',
 'Adventure',
 'Comedy',
 'Crime',
 'Drama',
 'Film-Noir',
 'Horror',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War'}

In [112]:
dict1 = {}
for i in genres_1:
    if i not in dict1:
        dict1[i] = 1
    else:
        dict1[i] = dict1[i] + 1
dict1

{'Action': 1, 'Drama': 1, 'History': 1, 'Thriller': 1, 'War': 1}

In [113]:
dict2 = {}
for i in genres_2:
    if i not in dict2:
        dict2[i] = 1
    else:
        dict2[i] = dict2[i] + 1
dict2

{'Action': 1, 'Drama': 1, 'Thriller': 1, 'Crime': 1}

In [114]:
director1_lst =[]
director2_lst =[]
for key,value in dict1.items():
    for key1,value1 in dict2.items():
        if key == key1:
            director1_lst.append(value)
            
for key,value in dict2.items():
    for key1,value1 in dict1.items():
        if key == key1:
            director2_lst.append(value)

In [115]:
director1_lst 

[1, 1, 1]

In [116]:
director2_lst

[1, 1, 1]

In [117]:
def consine_similarity(director1_lst,director2_lst  ):
    sum=0
    prod1 = 0
    prod2=0
    for i in range(len(director1_lst)):
        num = sum + (director1_lst[i]*director2_lst[i])
        prod1 = prod1+ (director1_lst[i])**2
        prod2 = prod2+ (director2_lst[i])**2
        prod1 = prod1**0.5
        prod2 = prod2**0.5
        den = prod1*prod2
        #print(den)
        similarity = num/den
    return similarity

In [118]:
consine_similarity(director1_lst ,director2_lst )

0.414213562373095